In [2]:
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Here's an outline of your project plan:
- Task: Collect and analyze weather data across cities worldwide.
- Purpose: PlanMyTrip will use the data to recommend ideal hotels based on clients' weather preferences.
- Method: Create a Pandas DataFrame with 500 or more of the world's unique cities and their weather data in real time. This process will entail collecting, analyzing, and visualizing the data.

Your analysis of the data will be split into three main parts, or stages.

1. **Collect the Data**

- Use the NumPy module to generate more than 1,500 random latitudes and longitudes.
- Use the citipy module to list the nearest city to the latitudes and longitudes.
- Use the OpenWeatherMap API to request the current weather data from each unique city in your list.
- Parse the JSON data from the API request.
- Collect the following data from the JSON file and add it to a DataFrame:
    - City, country, and date
    - Latitude and longitude
    - Maximum temperature
    - Humidity
    - Cloudiness
    - Wind speed

2. **Exploratory Analysis with Visualization**

- Create scatter plots of the weather data for the following comparisons:
    - Latitude versus temperature
    - Latitude versus humidity
    - Latitude versus cloudiness
    - Latitude versus wind speed
- Determine the correlations for the following weather data:
    - Latitude and temperature
    - Latitude and humidity
    - Latitude and cloudiness
    - Latitude and wind speed
- Create a series of heatmaps using the Google Maps and Places API that showcases the following:
    - Latitude and temperature
    - Latitude and humidity
    - Latitude and cloudiness
    - Latitude and wind speed
    
3. **Visualize Travel Data**
Create a heatmap with pop-up markers that can display information on specific cities based on a customer's travel preferences. Complete these steps:


1. Filter the Pandas DataFrame based on user inputs for a minimum and maximum temperature.
2. Create a heatmap for the new DataFrame.
3. Find a hotel from the cities' coordinates using Google's Maps and Places API, and Search Nearby feature.
4. Store the name of the first hotel in the DataFrame.
5. Add pop-up markers to the heatmap that display information about the city, current maximum temperature, and a hotel in the city.

# Generate Random Coordinates of World Cities

We use the **geographic coordinate system** (**GCS**) to reference any point on Earth by its latitude and longitude coordinates.

**Latitudes** are imaginary lines on Earth that run parallel east to west and are measured in angular units called degrees, minutes, and seconds, with 60 minutes in a degree and 60 seconds in a minute. Sometimes a latitude is referred to as a **parallel**.

The **equator** is an imaginary line around the middle of the earth that is equidistant from the North and South Poles and has a latitude of 0°. The equator splits Earth into Northern and Southern Hemispheres.

> All latitude lines above the equator are measured northward and considered positive, after 0° (the equator) and up to 90°, or 90° north (the North Pole). All latitude lines below the equator are measured southward and considered negative, before 0° (the equator) and down to -90°, or 90° south (the South Pole).

**Longitudes** are imaginary lines on Earth that run from the North to the South Poles and are called **meridians**. The prime meridian represents zero meridian, the origin for longitude coordinates, and splits Earth into the Eastern and Western Hemispheres.

> All meridians east of the prime meridian are considered positive, after 0° and up to 180°. All meridians west of the prime meridian are considered negative, before 0° and down to –180°.

## Generate Random Latitudes and Longitudes

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-25.563496171544614 96.7547447064697
-36.19943365925268 -1.1656150041391413
86.4036787632408 -0.3361690552677885
73.96676823563004 39.83162993887916
64.78360926841492 -86.99372190077156
-41.20366794294728 51.890607369754434
39.635491849396516 -93.4797658926115
29.355077636953453 -81.09466138110314
-11.417972028567576 156.06296113147948
-11.937586989927553 -69.47129134154193
52.15607700672075 64.77315559274157
-6.67695397945262 -61.84906820465264
10.25901505515418 169.68882080442927
-25.032103372945087 -54.87449815171817
59.29596766862724 -78.46132007366508
-53.369090374563704 172.46217504765144
22.23326786255049 -127.63944586699645
83.75491296046869 149.11964060989533
22.388631449026732 104.99725246564537
-64.32041910974704 -83.14955525963268
75.94562331523338 4.97747740322734
63.1027967696684 37.74976438432742
47.086378408346604 -115.28604574406953
-47.420741862341025 -53.41535419506174
-24.645957203085473 103.19706687013229
52.86638857663894 29.30865280062423
55.68258729920694 -148.3

## Generate Random World Cities